# Gerador de notícias rotuladas pela API do CHAT GPT

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG
- Trabalho de conclusão de curso - MBA Digital Business USP Esalq

In [9]:
import openai
import os
import pandas as pd
import time
import re
import os

openai.api_key = os.getenv('GPT_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]


def classifica_sentimento_gpt(texto, model='gpt-3.5-turbo'):
    response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {
                      "role": "system",
                      "content": "Você receberá um texto de uma notícia sobre uma empresa, e deverá classificá-la como positiva, negativa ou neutra"
                    },
                    {
                      "role": "user",
                      "content": texto
                    }
                  ],
                temperature=0,
                max_tokens=256,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0
                )
    return response.choices[0].message["content"]


def trata_noticias(response, dimensao, sentimento, unica=False):
    response = response.replace('Matéria ', '\n\n\nMatéria ' )
    response = response.replace('\n\n\nMatéria 1', 'Matéria 1' )
    response = response.replace('Entrevista ', '\n\n\nEntrevista ' )
    response = response.replace('\n\n\nEntrevista 1', 'Entrevista 1' )

    sep = '\n'
    if ('\n\n' in response):
        sep = '\n\n'
    if ('\n\n\n' in response):
        sep = '\n\n\n'
    if ('#' in response):
        sep = '#'
    df=None
    
    if unica:
        df = pd.DataFrame({'Texto' : [response], 'Dimensão' : [dimensao]  ,
                     'Sentimento' : [sentimento]  })
    else:
        df = pd.DataFrame({'Texto' : response.split(sep), 'Dimensão' : [dimensao] * len(response.split(sep)) ,
                     'Sentimento' : [sentimento] * len(response.split(sep)) })
    
    df = df.reset_index(drop=True)
    df = df.drop_duplicates()
    df = df[  (~pd.isnull(df['Texto']))  &   (df['Texto'].str.strip() != '')]
    df['Texto'] = df['Texto'].str.replace('Empresa [A-Z]{1,3} ', 'Empresa ', regex=True)
    df['Texto'] = df['Texto'].str.replace('Texto [0-9]{1,2}: ', '', regex=True)
    df['Texto'] = df['Texto'].str.replace(r'^[0-9]{1,2}\. ', '', regex=True)
    df['Texto'] = df['Texto'].str.replace(r'(Continua...)', '')
    df['Texto'] = df['Texto'].str.replace('\n\nMatéria ', 'Matéria ')
    df['Texto'] = df['Texto'].str.replace('Matéria [0-9]{1,2}: ', '', regex=True)
    df['Texto'] = df['Texto'].str.replace('Matéria[0-9]{1,2}: ', '', regex=True)
    df['Texto'] = df['Texto'].str.replace('\n\nEntrevista ', 'Entrevista ')
    df['Texto'] = df['Texto'].str.replace('Entrevista [0-9]{1,2}: ', '', regex=True)
    df['Texto'] = df['Texto'].str.replace('Entrevista [0-9]{1,2}:\n\n', '', regex=True)
    df['Texto'] = df['Texto'].str.replace('Entrevista[0-9]{1,2}: ', '', regex=True)
    df['Texto'] = df['Texto'].str.replace('\n\nMatéria ', 'Matéria ')

    return df.drop_duplicates()

                              
def obtem_noticias(texto, dimensao, sentimento, model='gpt-3.5-turbo', unica=False):
    return trata_noticias(get_completion(texto, 'gpt-4'), dimensao, sentimento, unica)


## Lendo a base já gravada

In [10]:
df = pd.read_excel('datasets/gpt_noticias_ESG.xlsx')

## Buscando as notícias - rodar 1 prompt por vez

In [504]:
#prompt, dimensao, sentimento = 'Gere 10 notícias fictícias de mais de 3000 caracteres sobre empresas que estão lançando produtos. Separe cada notícia por 1 linha apenas. Todas as notícias devem ser positivas.', 'Outros', 'Positiva'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere apenas uma notícia fictícia e negativa de mais de 3000 caracteres sobre uma empresa e a dimensão ambiental. ', 'E', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=True)

#prompt, dimensao, sentimento = 'Gere apenas uma notícia fictícia e negativa de mais de 3000 caracteres sobre uma empresa e a dimensão social. ', 'S', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=True)

#prompt, dimensao, sentimento = 'Gere 10 notícias fictícias e negativas de mais de 3000 caracteres sobre uma empresa e a dimensão ambiental. Separa cada notícia por 2 linhas em branco', 'E', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere 19 notícias fictícias e positivas de mais de 3000 caracteres sobre uma empresa e a dimensão de governança. Separa cada notícia por 2 linhas em branco', 'G', 'Positiva'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere apenas uma notícia fictícia e negativa de mais de 3000 caracteres sobre o fracasso de uma campanha de vendas. ', 'Outros', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=True)

#prompt, dimensao, sentimento = 'Gere 20 notícias negativas longas sobre uma empresa que está sendo investigada ou foi multada pelo ministério do trabalho ou ministério público do trabalho por suas práticas trabalhistas inadequadas. Separa cada notícias por duas linhas em branco. ', 'S', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere exatamente 10 notícias negativas longas sobre uma empresa que está sendo acusada de greenwashing. Separe cada notícias por duas linhas em branco. ', 'E', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere exatamente 10 notícias negativas de mais de 3000 caracteres sobre uma empresa que foi multada por eventos ambientais. Separe cada notícias por duas linhas em branco. ', 'E', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere exatamente 10 notícias negativas de mais de 3000 caracteres sofreu um incidente cibernético, comprometendo os dados de seus clientes', 'G', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere exatamente 6 notícias positivas de mais de 3000 caracteres sobre uma empresa que está implementando açoes para evoluir a dimensão G do ESG.  Separe cada notícias por duas linhas em branco. ', 'G', 'Positiva'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere exatamente 10 notícias positivas de mais de 3000 caracteres sobre entrevistas de CEOs de empresas brasileiras sobre o cenário econômico do Brasil e do mundo.  Separe cada notícias por duas linhas em branco. ', 'Outros', 'Positiva'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere exatamente 10 notícias negativas de mais de 3000 caracteres sobre entrevistas de CEOs de empresas brasileiras sobre o cenário econômico do Brasil e do mundo.  Separe cada notícias por duas linhas em branco. ', 'Outros', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere exatamente 5 notícias positivas de mais de 3000 caracteres sobre entrevistas de CEOs de empresas brasileiras sobre o cenário político do Brasil.  Separe cada notícias por duas linhas em branco. ', 'Outros', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)


#prompt, dimensao, sentimento = 'Gere exatamente 10 notícias negativas de mais de 3000 caracteres sobre o falecimento de ex-executivos de empresas.  Separe cada notícia na resposta pelo símbolo #', 'Outros', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)


#prompt, dimensao, sentimento = 'Gere exatamente 10 notícias positivas de mais de 3000 caracteres sobre ações de marketing de empresas no Big Brother Brasil ou Rock in Rio. Não comentar sobre sustentabilidade e diversidade. Separe cada notícia na resposta pelo símbolo #', 'Outros', 'Positiva'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere exatamente 10 notícias positivas de mais de 3000 caracteres sobre entrevistas de empresários brasileiros sobre o ambiente político e econômico. Não comentar sobre sustentabilidade e diversidade. Separe cada notícia na resposta pelo símbolo #', 'Outros', 'Positiva'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere exatamente 10 notícias negativas de mais de 3000 caracteres sobre entrevistas de empresários brasileiros sobre o ambiente político e econômico. Não comentar sobre sustentabilidade e diversidade. Separe cada notícia na resposta pelo símbolo #', 'Outros', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere exatamente 5 notícias positivas de mais de 3000 caracteres sobre empresas que estão inaugurando lojas e canais digitais. Não comentar sobre sustentabilidade e diversidade. Separe cada notícia na resposta pelo símbolo #', 'Outros', 'Positiva'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

#prompt, dimensao, sentimento = 'Gere exatamente 5 notícias negativas de mais de 3000 caracteres sobre empresas que estão inaugurando lojas e canais digitais. Não comentar sobre sustentabilidade e diversidade. Separe cada notícia na resposta pelo símbolo #', 'Outros', 'Negativa'
#df_noticias = obtem_noticias(prompt, dimensao, sentimento, unica=False)

# social
#sent_query = 'positivas'
#dimensao = 'S'
#prompt = 'Gere exatamente 10 notícias ' + sent_query + ' de mais de 3000 caracteres sobre empresas que estão realizando ações no campo social e de diversidade. Separe cada notícia na resposta pelo símbolo #'
#df_noticias = obtem_noticias(prompt, dimensao, sent_query, unica=False)

#sent_query = 'negativas'
#dimensao = 'S'
#prompt = 'Gere exatamente 10 notícias ' + sent_query + ' de mais de 3000 caracteres sobre empresas que estão envolvidas em controvérsias no campo social e de diversidade. Separe cada notícia na resposta pelo símbolo #'
#df_noticias = obtem_noticias(prompt, dimensao, sent_query, unica=False)

# governanca
#sent_query = 'positivas'
#dimensao = 'G'
#prompt = 'Gere exatamente 10 notícias ' + sent_query + ' de mais de 3000 caracteres sobre empresas que estão realizando ações positivas no campo da governança corporativa. Não falar sobre diversidade e sustentabilidade. Separe cada notícia na resposta pelo símbolo #'
#df_noticias = obtem_noticias(prompt, dimensao, sent_query, unica=False)

#sent_query = 'negativas'
#dimensao = 'G'
#prompt = 'Gere exatamente 10 notícias ' + sent_query + ' de mais de 3000 caracteres sobre empresas que estão envolvidas em controvérsias sobre governança corporativa. Não falar sobre diversidade e sustentabilidade. Separe cada notícia na resposta pelo símbolo #'
#df_noticias = obtem_noticias(prompt, dimensao, sent_query, unica=False)

# Ambiental
#sent_query = 'positivas'
#dimensao = 'E'
#prompt = 'Gere exatamente 10 notícias ' + sent_query + ' de mais de 3000 caracteres sobre empresas que estão realizando ações positivas no campo ambiental. Separe cada notícia na resposta pelo símbolo #'
#df_noticias = obtem_noticias(prompt, dimensao, sent_query, unica=False)

#sent_query = 'negativas'
#dimensao = 'E'
#prompt = 'Gere exatamente 10 notícias ' + sent_query + ' de mais de 3000 caracteres sobre empresas que estão envolvidas em controvérsias no campo ambiental. Separe cada notícia na resposta pelo símbolo #'
#df_noticias = obtem_noticias(prompt, dimensao, sent_query, unica=False)


df_noticias = df_noticias[df_noticias.Texto.str.len()>10].reset_index(drop=True)

df_noticias

/var/folders/cv/jnvn5ttn6qd82y07scl_zqpc0000gn/T/ipykernel_25496/3200863601.py:69: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Texto'] = df['Texto'].str.replace(r'(Continua...)', '')


,Texto,Dimensão,Sentimento
0,1) A gigante do petróleo BP enfrenta críticas ...,E,negativas
1,"\nA British Petroleum (BP), uma das maiores em...",E,negativas
2,"\nA Nestlé, uma das maiores empresas de alimen...",E,negativas
3,"\nA Volkswagen, uma das maiores fabricantes de...",E,negativas
4,"\nA Monsanto, uma das maiores empresas de agro...",E,negativas
5,"\nA Royal Dutch Shell, uma das maiores empresa...",E,negativas
6,"\nA Coca-Cola, uma das maiores empresas de beb...",E,negativas
7,"\nA Amazon, a maior empresa de comércio eletrô...",E,negativas
8,"\nA Chevron, uma das maiores empresas de petró...",E,negativas
9,"\nA Apple, uma das maiores empresas de tecnolo...",E,negativas


## Adicionando à base atual o resultado acima

In [506]:
df = pd.concat([df, df_noticias]).drop_duplicates().reset_index(drop=True)

## Classificando o sentimento do que não possui rótulos

In [507]:
df['Sentimento'] = df.apply(lambda row: 
                            classifica_sentimento_gpt(row['Texto'], model='gpt-4') if pd.isnull(row['Sentimento']) 
                            else row['Sentimento'], axis=1)

# ajustando nomenclatura
df.Sentimento = df.Sentimento.str.replace('positivas', 'Positiva') 
df.Sentimento = df.Sentimento.str.replace('negativas', 'Negativa') 
df.Sentimento = df.Sentimento.str.replace('Positivas', 'Positiva') 
df.Sentimento = df.Sentimento.str.replace('Negativas', 'Negativa') 
df.Sentimento = df.Sentimento.str.replace('positiva', 'Positiva') 
df.Sentimento = df.Sentimento.str.replace('negativa', 'Negativa') 

## Visualizando a base coletada

In [11]:
df.groupby(['Dimensão', 'Sentimento']).count()

Texto
Dimensão Sentimento       
E        Negativa       85
         Neutra          2
         Positiva      103
G        Negativa       95
         Neutra          1
         Positiva       94
Outros   Negativa       67
         Neutra         36
         Positiva      237
S        Negativa       81
         Neutra          1
         Positiva      108

In [12]:
df.groupby(['Dimensão']).count()

,Texto,Sentimento
Dimensão,,
E,190,190
G,190,190
Outros,340,340
S,190,190


In [13]:
df

,Texto,Dimensão,Sentimento
0,Empresa reduz emissões de carbono em 30% e rec...,E,Positiva
1,Empresa investe em energia renovável para redu...,E,Positiva
2,Empresa lança programa de reciclagem de resídu...,E,Positiva
3,Empresa adota práticas de agricultura sustentá...,E,Positiva
4,Empresa lança programa de reflorestamento para...,E,Positiva
...,...,...,...
905,"\nA Coca-Cola, uma das maiores empresas de beb...",E,Negativa
906,"\nA Amazon, a maior empresa de comércio eletrô...",E,Negativa
907,"\nA Chevron, uma das maiores empresas de petró...",E,Negativa
908,"\nA Apple, uma das maiores empresas de tecnolo...",E,Negativa


## Salvando o resultado 

In [14]:
df.to_excel('datasets/gpt_noticias_ESG.xlsx', index=False)

In [15]:
len(df)

910